# 近似训练（以skip-gram模型为例）

跳元模型的主要思想是使用softmax运算来计算基于给定的中心词wc生成上下文字wo的条件概率（如 (14.1.4)），对应的对数损失在 (14.1.7)给出。

由于softmax操作的性质，上下文词可以是词表V中的任意项， (14.1.7)包含与整个词表大小一样多的项的求和。因此， (14.1.8)中跳元模型的梯度计算和 (14.1.15)中的连续词袋模型的梯度计算都包含求和。不幸的是，在一个词典上（通常有几十万或数百万个单词）求和的梯度的计算成本是巨大的！

为了降低上述计算复杂度，本节将介绍两种近似训练方法：负采样和分层softmax。 由于跳元模型和连续词袋模型的相似性，我们将以跳元模型为例来描述这两种近似训练方法。

## 负采样

负采样修改了原目标函数。给定中心词的上下文窗口$w_c$，任意上下文词$w_o$来自该上下文窗口的被认为是由下式建模概率的事件：
$$P\left(D=1 \mid w_c, w_o\right)=\sigma\left(\mathbf{u}_o^{\top} \mathbf{v}_c\right)$$

****
理解：

在负采样（negative sampling）中，D是一个二进制随机变量，表示一个事件是否发生。在这个描述中，D=1表示事件发生，即给定中心词$w_c$和上下文词$w_o$，$w_o$被认为是来自$w_c$的上下文词。

公式$P(D=1|w_c, w_o)$表示给定中心词$w_c$和上下文词$w_o$的条件下，$w_o$被认为是来自$w_c$的上下文词的概率。这个概率通过一个sigmoid函数来建模，其中σ表示sigmoid函数。具体地，公式中的向量$u_o$和$v_c$分别表示与上下文词$w_o$和中心词$w_c$关联的参数向量。

因此，公式$P(D=1|w_c, w_o)$可以理解为在负采样中判断给定上下文窗口中的一个上下文词是否来自中心词的概率。

用 $S$ 表示上下文词 $w_o$ 来自中心词 $w_c$ 的上下文窗口的事件。对于这个涉及 $w_o$ 的事件, 从预定义分布 $P(w)$ 中采样 $K$ 个不是来自这个上下文窗口噪声词。用 $N_k$ 表示噪声词 $w_k(k=1, \ldots, K)$ 不是来自 $w_c$ 的上下文窗口的事件。假设正例和负例 $S, N_1, \ldots, N_K$ 的这些事件是相互独立的。负采样将 (14.2.3)中的联合概率（仅涉及正例）重写为
$$
\prod_{t=1}^T \prod_{-m \leq j \leq m, j \neq 0} P\left(w^{(t+j)} \mid w^{(t)}\right)
$$

通过事件 $S, N_1, \ldots, N_K$ 近似条件概率:
$$
P\left(w^{(t+j)} \mid w^{(t)}\right)=P\left(D=1 \mid w^{(t)}, w^{(t+j)}\right) \prod_{k=1, w_k \sim P(w)}^K P\left(D=0 \mid w^{(t)}, w_k\right)
$$

分别用 $i_t$ 和 $h_k$ 表示词 $w^{(t)}$ 和噪声词 $w_k$ 在文本序列的时间步 $t$ 处的索引。 (14.2.5)中关于条件概率的对 数损失为:
$$
\begin{aligned}
-\log P\left(w^{(t+j)} \mid w^{(t)}\right) & =-\log P\left(D=1 \mid w^{(t)}, w^{(t+j)}\right)-\sum_{k=1, w_k \sim P(w)}^K \log P\left(D=0 \mid w^{(t)}, w_k\right) \\
& =-\log \sigma\left(\mathbf{u}_{i_{t+j}^{\top}}^{\top} \mathbf{v}_{i_t}\right)-\sum_{k=1, w_k \sim P(w)}^K \log \left(1-\sigma\left(\mathbf{u}_{h_k}^{\top} \mathbf{v}_{i_t}\right)\right) \\
& =-\log \sigma\left(\mathbf{u}_{i_{t+j}}^{\top} \mathbf{v}_{i_t}\right)-\sum_{k=1, w_k \sim P(w)}^K \log \sigma\left(-\mathbf{u}_{h_k}^{\top} \mathbf{v}_{i_t}\right) .
\end{aligned}
$$

****
理解：

这段描述和公式是在解释负采样（Negative Sampling）方法中的联合概率表示方式。

在负采样中，我们考虑一个中心词$w_c$和一个上下文窗口中的上下文词$w_o$，以及从预定义分布$P(w)$中采样得到的$K$个噪声词$w_k$（不属于上下文窗口）。事件$S$表示$w_o$来自$w_c$的上下文窗口，事件$N_k$表示噪声词$w_k$不来自$w_c$的上下文窗口。

公式表示了联合概率的计算方式，即对于每个时刻$t$，都需要考虑$w^{(t+j)}$（$-m \leq j \leq m, j \neq 0$，即上下文窗口中的上下文词）的条件概率$P(w^{(t+j)}|w^{(t)})$。这样，整个联合概率可以表示为对所有时刻和上下文词的条件概率的乘积。

负采样的目标是通过最大化联合概率来优化模型，其中正例$S$的概率需要最大化，而负例$N_1, \ldots, N_K$的概率需要最小化。

简单来说，公式描述了负采样中正例和负例事件之间的独立性假设，并通过乘积形式将联合概率进行重写，以便进行优化。

## 层序softmax模型

作为另一种近似训练方法，层序Softmax（hierarchical softmax）使用二叉树（ 图14.2.1中说明的数据结构），其中树的每个叶节点表示词表V中的一个词。

![](https://files.mdnice.com/user/23981/ce8d37ea-6f42-4e93-8477-d4a8e8766d14.png)

用$L(w)$表示二叉树中表示字$w$的从根节点到叶节点的路径上的节点数（包括两端）。设$n(w, j)$为该路径上的  $j^{\text {th }}$节点, 其上下文字向量为$\mathbf{u}_{n(w, j)}$。例如, 图14.2.1中的$L\left(w_{3}\right)=4$。分层softmax将 (14.1.4)中的条件概率近似为

$$P\left(w_{o} \mid w_{c}\right)=\prod_{j=1}^{L\left(w_{o}\right)-1} \sigma\left(\llbracket n\left(w_{o}, j+1\right)=\operatorname{leftChild}\left(n\left(w_{o}, j\right)\right) \rrbracket \cdot \mathbf{u}_{n\left(w_{o}, j\right)}^{\top} \mathbf{v}_{c}\right)$$

其中函数$\sigma$在 (14.2.2)中定义,$\operatorname{leftChild}(n)$是节点n的左子节点：如果x为真,  $\llbracket x \rrbracket=1$; 否则$\llbracket x \rrbracket=-1_{\circ}$

****
理解：

这段描述和公式是在解释分层Softmax（Hierarchical Softmax）方法中的条件概率计算方式。

在分层Softmax中，使用二叉树来表示词汇表，树的每个节点表示一个词，根节点表示整个词汇表，叶节点表示具体的词。假设$L(w)$表示表示词$w$在树中的路径上的节点数（包括起始节点和结束节点），$n(w, j)$表示路径上的第$j$个节点，$\mathbf{u}_{n(w, j)}$表示该节点的上下文向量。

公式描述了分层Softmax中的条件概率计算方式。对于给定的中心词$w_c$和上下文词$w_o$，我们要计算$P(w_o|w_c)$的概率。公式中的乘积表示从路径上的第一个节点到倒数第二个节点的条件概率的连乘。

在连乘的每一项中，$\sigma(\cdot)$表示sigmoid函数，它用于将内积的结果转化为概率。$\llbracket n(w_o, j+1) = \operatorname{leftChild}(n(w_o, j)) \rrbracket$表示如果$n(w_o, j+1)$是$n(w_o, j)$的左子节点，则为1；否则为-1。

简单来说，公式描述了分层Softmax中通过二叉树路径上的节点来计算条件概率的方式。通过将路径上的条件概率进行连乘，可以近似计算给定中心词和上下文词的概率。

在分层Softmax中，左子节点被定义为1，右子节点被定义为-1是为了方便计算和表示。这是一种约定俗成的选择，并没有特别的理论依据。

通过将节点标记为1和-1，可以将条件概率的计算表示为一个累乘的过程。每个节点对应的向量乘以中心词向量的内积，然后通过sigmoid函数将其转换为概率。累乘的过程保证了路径上所有节点的条件概率相乘，最终得到整个条件概率。

分层Softmax相对于传统的训练方法速度更快的原因是，它减少了计算和更新的复杂度。在传统的训练方法中，需要对整个词汇表的每个词进行计算和更新，而在分层Softmax中，我们只需要遍历树的路径上的节点。由于二叉树的结构，树的高度相对较小，因此计算和更新的复杂度大大降低。这种减少计算和更新的复杂度的优势使得分层Softmax在大规模词汇表和大型语料库上具有更高的效率。